# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2053530400249,
 'week52high': 150.76,
 'week52low': 59.07,
 'week52highSplitAdjustOnly': 149.6,
 'week52lowSplitAdjustOnly': 60.34,
 'week52change': 0.9516292287305769,
 'sharesOutstanding': 17021976644,
 'float': 0,
 'avg10Volume': 96107234,
 'avg30Volume': 116989815,
 'day200MovingAvg': 125.8,
 'day50MovingAvg': 129.92,
 'employees': 147719,
 'ttmEPS': 3.79,
 'ttmDividendRate': 0.8238750906455431,
 'dividendYield': 0.007062727717194121,
 'nextDividendDate': '0',
 'exDividendDate': '2021-02-03',
 'nextEarningsDate': '2021-01-19',
 'peRatio': 32.22963317744361,
 'beta': 1.256867349267256,
 'maxChangePercent': 47.6777790883754,
 'year5ChangePercent': 3.956169416813198,
 'year2ChangePercent': 1.646465551929306,
 'year1ChangePercent': 1.0001860575733221,
 'ytdChangePercent': -0.08530668953846952,
 'month6ChangePercent': 0.0680170716841691,
 'month3ChangePercent': -0.09969814955481887,
 'month1ChangePercent': 0.0011068625510953392,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

1.0001860575733221

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=my_columns),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,129.070,0.792015,N/A
1,AAL,24.868,0.660253,N/A
2,AAP,189.713,0.99458,N/A
3,AAPL,120.350,1.019146,N/A
4,ABBV,109.520,0.55901,N/A
...,...,...,...,...
500,YUM,112.570,0.639875,N/A
501,ZBH,165.920,0.701332,N/A
502,ZBRA,475.090,1.561852,N/A
503,ZION,57.350,1.045886,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,33.710,4.350751,N/A
1,275,LB,62.910,3.902346,N/A
2,208,GPS,30.906,2.726154,N/A
3,462,URI,342.030,2.713044,N/A
4,148,DVN,21.940,2.69283,N/A
5,467,VIAC,49.530,2.664703,N/A
6,212,HAL,22.350,2.376279,N/A
7,272,KSS,61.300,2.36011,N/A
8,314,MOS,32.122,2.294237,N/A
9,175,FANG,76.930,2.271196,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
portfolio_size

'10000000'

In [11]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

200000.0
c:\workspace\algorithmic-trading-python\.virtualenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,33.710,4.350751,5932
1,275,LB,62.910,3.902346,3179
2,208,GPS,30.906,2.726154,6471
3,462,URI,342.030,2.713044,584
4,148,DVN,21.940,2.69283,9115
5,467,VIAC,49.530,2.664703,4037
6,212,HAL,22.350,2.376279,8948
7,272,KSS,61.300,2.36011,3262
8,314,MOS,32.122,2.294237,6226
9,175,FANG,76.930,2.271196,2599


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.570,N/A,0.813647,N/A,0.263477,N/A,0.071912,N/A,0.028577,N/A,N/A
1,AAL,24.520,N/A,0.632695,N/A,0.87794,N/A,0.454626,N/A,0.095781,N/A,N/A
2,AAP,191.145,N/A,0.957028,N/A,0.233167,N/A,0.183021,N/A,0.162211,N/A,N/A
3,AAPL,125.040,N/A,1.009218,N/A,0.069649,N/A,-0.101173,N/A,0.001113,N/A,N/A
4,ABBV,108.380,N/A,0.573491,N/A,0.263062,N/A,0.032136,N/A,-0.009749,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.350,N/A,0.6378,N/A,0.225622,N/A,0.00262,N/A,0.056215,N/A,N/A
501,ZBH,168.170,N/A,0.726201,N/A,0.206397,N/A,0.053136,N/A,-0.016761,N/A,N/A
502,ZBRA,480.550,N/A,1.582827,N/A,0.826214,N/A,0.24679,N/A,-0.066008,N/A,N/A
503,ZION,55.950,N/A,1.046397,N/A,0.896226,N/A,0.279461,N/A,0.013844,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

hqm_dataframe.fillna(value=0, inplace=True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.570,N/A,0.813647,63.168317,0.263477,48.316832,0.071912,36.435644,0.028577,32.277228,N/A
1,AAL,24.520,N/A,0.632695,47.920792,0.877940,94.851485,0.454626,97.623762,0.095781,65.148515,N/A
2,AAP,191.145,N/A,0.957028,74.059406,0.233167,43.168317,0.183021,66.138614,0.162211,92.871287,N/A
3,AAPL,125.040,N/A,1.009218,76.831683,0.069649,17.623762,-0.101173,2.574257,0.001113,20.39604,N/A
4,ABBV,108.380,N/A,0.573491,43.366337,0.263062,47.920792,0.032136,23.168317,-0.009749,15.247525,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.350,N/A,0.637800,48.514851,0.225622,42.574257,0.002620,18.019802,0.056215,45.346535,N/A
501,ZBH,168.170,N/A,0.726201,57.029703,0.206397,40.0,0.053136,29.50495,-0.016761,13.861386,N/A
502,ZBRA,480.550,N/A,1.582827,93.861386,0.826214,93.069307,0.246790,81.386139,-0.066008,5.148515,N/A
503,ZION,55.950,N/A,1.046397,79.207921,0.896226,95.643564,0.279461,85.742574,0.013844,25.742574,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [14]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.570,N/A,0.813647,63.168317,0.263477,48.316832,0.071912,36.435644,0.028577,32.277228,45.049505
1,AAL,24.520,N/A,0.632695,47.920792,0.877940,94.851485,0.454626,97.623762,0.095781,65.148515,76.386139
2,AAP,191.145,N/A,0.957028,74.059406,0.233167,43.168317,0.183021,66.138614,0.162211,92.871287,69.059406
3,AAPL,125.040,N/A,1.009218,76.831683,0.069649,17.623762,-0.101173,2.574257,0.001113,20.39604,29.356436
4,ABBV,108.380,N/A,0.573491,43.366337,0.263062,47.920792,0.032136,23.168317,-0.009749,15.247525,32.425743
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.350,N/A,0.637800,48.514851,0.225622,42.574257,0.002620,18.019802,0.056215,45.346535,38.613861
501,ZBH,168.170,N/A,0.726201,57.029703,0.206397,40.0,0.053136,29.50495,-0.016761,13.861386,35.09901
502,ZBRA,480.550,N/A,1.582827,93.861386,0.826214,93.069307,0.246790,81.386139,-0.066008,5.148515,68.366337
503,ZION,55.950,N/A,1.046397,79.207921,0.896226,95.643564,0.279461,85.742574,0.013844,25.742574,71.584158


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [15]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm50_dataframe = hqm_dataframe[:50]
hqm50_dataframe.reset_index(drop=True, inplace=True)
hqm50_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NUE,82.680,N/A,1.508342,92.871287,0.774774,90.49505,0.548857,99.207921,0.320645,100.0,95.643564
1,GPS,31.174,N/A,2.776158,99.60396,0.738715,88.316832,0.436023,97.029703,0.154905,91.089109,94.009901
2,OKE,50.760,N/A,1.797569,95.841584,1.035558,97.623762,0.373072,93.861386,0.142829,87.128713,93.613861
3,EOG,76.320,N/A,1.151323,83.168317,1.100673,97.821782,0.526641,99.009901,0.144132,88.118812,92.029703
4,HIG,69.000,N/A,0.960537,74.851485,0.872140,94.455446,0.412137,96.435644,0.311802,99.80198,91.386139
5,LB,63.809,N/A,3.933191,99.80198,0.973766,96.435644,0.608381,99.60396,0.096409,66.336634,90.544554
6,URI,337.900,N/A,2.647820,99.207921,0.878666,95.049505,0.433128,96.831683,0.091536,63.168317,88.564356
7,FANG,76.330,N/A,2.287862,98.415842,1.491615,99.009901,0.614985,99.80198,0.078214,56.831683,88.514851
8,IVZ,25.880,N/A,1.742242,95.247525,1.221370,98.613861,0.446916,97.425743,0.087870,61.386139,88.168317
9,EXPE,181.410,N/A,1.953008,96.831683,0.946986,96.237624,0.342697,92.277228,0.095926,65.742574,87.772277


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [16]:
portfolio_input()

In [17]:
position_size = float(portfolio_size) / len(hqm50_dataframe.index)
for i in hqm50_dataframe.index:
    hqm50_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm50_dataframe.loc[i, 'Price'])

hqm50_dataframe

c:\workspace\algorithmic-trading-python\.virtualenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NUE,82.680,2418,1.508342,92.871287,0.774774,90.49505,0.548857,99.207921,0.320645,100.0,95.643564
1,GPS,31.174,6415,2.776158,99.60396,0.738715,88.316832,0.436023,97.029703,0.154905,91.089109,94.009901
2,OKE,50.760,3940,1.797569,95.841584,1.035558,97.623762,0.373072,93.861386,0.142829,87.128713,93.613861
3,EOG,76.320,2620,1.151323,83.168317,1.100673,97.821782,0.526641,99.009901,0.144132,88.118812,92.029703
4,HIG,69.000,2898,0.960537,74.851485,0.872140,94.455446,0.412137,96.435644,0.311802,99.80198,91.386139
5,LB,63.809,3134,3.933191,99.80198,0.973766,96.435644,0.608381,99.60396,0.096409,66.336634,90.544554
6,URI,337.900,591,2.647820,99.207921,0.878666,95.049505,0.433128,96.831683,0.091536,63.168317,88.564356
7,FANG,76.330,2620,2.287862,98.415842,1.491615,99.009901,0.614985,99.80198,0.078214,56.831683,88.514851
8,IVZ,25.880,7727,1.742242,95.247525,1.221370,98.613861,0.446916,97.425743,0.087870,61.386139,88.168317
9,EXPE,181.410,1102,1.953008,96.831683,0.946986,96.237624,0.342697,92.277228,0.095926,65.742574,87.772277


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [35]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm50_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [36]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [37]:
column_formats = {
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Number of Shares to Buy', integer_template], 
    'D': ['One-Year Price Return', percent_template], 
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0],  column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy: